In [13]:
import numpy as np
from numpy.polynomial import Polynomial
import pandas as pd 
from sklearn.impute import SimpleImputer
from sklearn import svm
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

from copy import deepcopy

In [14]:
# from skorch.callbacks import EpochScoring
# from skorch import NeuralNetClassifier

In [15]:
train_features = pd.read_csv('../Task2/train_features.csv')
ground_truth = pd.read_csv('../Task2/train_labels.csv')
test_features = pd.read_csv('../Task2/test_features.csv')

In [16]:
pids = train_features["pid"].unique().astype(np.int)
print('number of validation patients:{}'.format(pids.size))
print('number of prediction patients:{}'.format(test_features.pid.unique().size))

number of validation patients:18995
number of prediction patients:12664


Pre Processing

In [17]:
training_mean = {}
training_std = {}
training_med = {}
for key in train_features:
    #print(key)
    training_mean[key]=np.nanmean(train_features[key])
    training_std[key]=np.nanstd(train_features[key])
    training_med[key]=np.nanmedian(train_features[key])

In [18]:
train_features

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,3,34.0,NaN,NaN,12.0,NaN,36.0,8.7,24.0,...,NaN,100.0,NaN,114.0,24.6,94.0,NaN,NaN,142.0,7.33
1,1,4,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,99.0,NaN,NaN,125.0,7.33
2,1,5,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,92.0,NaN,NaN,110.0,7.37
3,1,6,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,104.0,7.37
4,1,7,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,22.4,81.0,NaN,NaN,100.0,7.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227935,9999,8,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,80.0,NaN,NaN,110.0,NaN
227936,9999,9,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,83.0,NaN,NaN,123.0,NaN
227937,9999,10,85.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,98.0,NaN,NaN,NaN,80.0,NaN,NaN,138.0,NaN
227938,9999,11,85.0,NaN,NaN,NaN,NaN,NaN,10.2,NaN,...,NaN,98.0,NaN,NaN,31.0,75.0,NaN,NaN,125.0,NaN


In [19]:
#Fill in the missing data using an imputer here i just use the median data
imputer = SimpleImputer(strategy='median').fit(train_features)
imputed_data= imputer.transform(train_features)
imputed_test= imputer.transform(test_features)

Subtask 1

In [20]:
labels1 = [
    "LABEL_BaseExcess",
    "LABEL_Fibrinogen",
    "LABEL_AST",
    "LABEL_Alkalinephos",
    "LABEL_Bilirubin_total",
    "LABEL_Lactate",
    "LABEL_TroponinI",
    "LABEL_SaO2",
    "LABEL_Bilirubin_direct",
    "LABEL_EtCO2"
]
ytrain = ground_truth[labels1].to_numpy()

In [21]:
#svc = svm.LinearSVC(C=1)

In [22]:
# for i,label in enumerate(ground_truth):
#   estimator = Pipline([('imputer',Imputer(missing_values = np.nan, strategy = 'median'])),
#                       ('')
#   scores = 

In [23]:
# for i,label in enumerate(ground_truth):
#   svc.fit(imputed_data,ytrain[:,i])

In [24]:
print(ground_truth.shape)
print(imputed_data.shape)
num_ground_truth = ground_truth.shape[0]
num_features = imputed_data.shape[1] - 2
num_hours = 12
imputed_data_slope_intersect = np.zeros((num_ground_truth, num_features*2+1))
imputed_data_slope_intersect.shape

generated_models = []

(18995, 16)
(227940, 37)


In [25]:
%%time
for i in range(num_ground_truth):
  current_hour = imputed_data[num_hours*i:num_hours*(i+1), 1]
  for j in range(num_features):
    intersect, slope = Polynomial.fit(current_hour, imputed_data[num_hours*i:num_hours*(i+1), j+2], deg=1).coef
    imputed_data_slope_intersect[i, 0]       = current_hour[-1] # The total time of stay
    imputed_data_slope_intersect[i, 2*j+1]   = intersect
    imputed_data_slope_intersect[i, 2*(j+1)] = slope
  #imputed_data[num_hours*i:num_hours*(i+1), :]

KeyboardInterrupt: 

In [ ]:
%%time
#clf = svm.SVR(kernel='linear')
model_task1 = Ridge(alpha=1e0, fit_intercept=True)

for label in labels1:
  scores = cross_val_score(model_task1, imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='roc_auc')
  print(scores)

[0.77160165 0.77468988 0.75738023 0.77933115 0.77019883]
[0.74074328 0.73177967 0.72535109 0.72433562 0.7386048 ]
[0.68534232 0.69095966 0.67612538 0.69031552 0.68307851]
[0.68821861 0.69469721 0.67829438 0.69172856 0.68472459]
[0.68398161 0.69324602 0.68462074 0.68837368 0.68323177]
[0.70854965 0.71185072 0.686267   0.71462721 0.68838198]
[0.76753436 0.75718505 0.73851685 0.76420241 0.73597975]
[0.73564189 0.73475077 0.72356891 0.7340884  0.73113097]
[0.73975789 0.72527576 0.66184969 0.69251142 0.75289854]
[0.78048705 0.80131622 0.7369809  0.77119497 0.80523364]
CPU times: user 2.91 s, sys: 4.83 s, total: 7.74 s
Wall time: 1.18 s


In [26]:
%%time 
from sklearn.ensemble import GradientBoostingClassifier
model_task1_gradient = GradientBoostingClassifier()
for label in labels1:
  scores = cross_val_score(model_task1_gradient,imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='roc_auc')
  print(scores )

KeyboardInterrupt: 

In [28]:
%%time 
from sklearn.ensemble import GradientBoostingRegressor
model_task1_gradient = GradientBoostingRegressor()
for label in labels1:
  scores = cross_val_score(model_task1_gradient,imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='roc_auc')
  print(scores )

[0.90621843 0.82420514 0.5        0.5        0.5       ]
[0.76996316 0.70295108 0.5        0.5        0.5       ]
[0.69126305 0.65850555 0.5        0.5        0.5       ]
[0.69401645 0.65662205 0.5        0.5        0.5       ]
[0.69728986 0.66046452 0.5        0.5        0.5       ]
[0.77429816 0.73980213 0.5        0.5        0.5       ]
[0.85931029 0.81041005 0.5        0.5        0.5       ]
[0.81596746 0.75046709 0.5        0.5        0.5       ]
[0.67657229 0.68343092 0.5        0.5        0.5       ]
[0.89994245 0.81772815 0.5        0.5        0.5       ]
Wall time: 4min 18s


In [40]:
for label in labels1:
  #model_task1 = Ridge(alpha=1e0, fit_intercept=True)
  # generated_models.append(deepcopy(model_task1.fit(imputed_data_slope_intersect, ground_truth[label])))
    fitted = model_task1_gradient.fit(imputed_data_slope_intersect, ground_truth[label])
    proba = fitted.predict_proba(imputed_test)
    print(proba)
    generated_models.append(deepcopy(proba))

In [29]:
%%time 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model_task1_gradient = make_pipeline(StandardScaler(),
                                    GradientBoostingClassifier()
)
for label in labels1:
  scores = cross_val_score(model_task1_gradient,imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='roc_auc',n_jobs=4,verbose = 2)
  print(scores )

Wall time: 26.9 ms


In [35]:
%%time 
model_task1_gradient = make_pipeline(StandardScaler(),
                                    GradientBoostingClassifier()
)
for label in labels1:
  fit = model_task1_gradient.fit(imputed_data_slope_intersect, ground_truth[label])
  probs = fit.predict_proba(imputed_data_slope_intersect)
  print(probs[:,0] )


[0.39261765 0.95264628 0.77402393 ... 0.31424701 0.95252234 0.73389247]
[0.95174028 0.96728192 0.96539795 ... 0.97059429 0.95390112 0.95099087]
[0.93032395 0.82675585 0.87006844 ... 0.78843515 0.89556088 0.84523   ]
[0.93545924 0.81790098 0.88463834 ... 0.81298152 0.8795885  0.84130376]


KeyboardInterrupt: 

In [32]:
probs[:,0]

array([0.95174028, 0.96728192, 0.96539795, ..., 0.97059429, 0.95390112,
       0.95099087])

In [14]:
for label in labels1:
  #model_task1 = Ridge(alpha=1e0, fit_intercept=True)
  # generated_models.append(deepcopy(model_task1.fit(imputed_data_slope_intersect, ground_truth[label])))
    generated_models.append(deepcopy(model_task1_gradient.fit(imputed_data_slope_intersect, ground_truth[label])))

In [44]:
labels2 = ["LABEL_Sepsis"]

In [ ]:
%%time
#clf = svm.SVR(kernel='linear')
model_task2 = Ridge(alpha=1e-3, fit_intercept=False)
for label in labels2:
  scores = cross_val_score(model_task2, imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='roc_auc')
  print(scores)

[0.60528422 0.62615497 0.63152265 0.67432257 0.62600434]
CPU times: user 344 ms, sys: 529 ms, total: 872 ms
Wall time: 122 ms


In [45]:
%%time 
from sklearn.ensemble import GradientBoostingClassifier
model_task2_gradient = GradientBoostingClassifier()
for label in labels2:
  scores = cross_val_score(model_task2_gradient,imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='roc_auc')
  print(scores )

[0.67847494 0.70410167 0.72232335 0.73375217 0.697787  ]
Wall time: 50.6 s


In [49]:
for label in labels2:
  #model_task2 = Ridge(alpha=1e-3, fit_intercept=False)
  # generated_models.append(deepcopy(model_task2.fit(imputed_data_slope_intersect, ground_truth[label])))
  generated_models.append(deepcopy(model_task2_gradient.fit(imputed_data_slope_intersect, ground_truth[label])))

Subtask 3

In [46]:
labels3 = ["LABEL_RRate", "LABEL_ABPm", "LABEL_SpO2", "LABEL_Heartrate"]

In [ ]:
%%time
#clf = svm.SVR(kernel='linear')
model_task3 = Ridge(alpha=5e4, fit_intercept=False)
for label in labels3:
  scores = cross_val_score(model_task3, imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='r2')
  print(scores)

[0.40894774 0.4127732  0.38678359 0.39499773 0.39439058]
[0.59394455 0.61273909 0.61196354 0.55735943 0.58479061]
[0.32619909 0.28476361 0.2387217  0.28472182 0.27133392]
[0.64062868 0.63574534 0.61505676 0.62174256 0.65007918]
Wall time: 511 ms


In [47]:
%%time 
from sklearn.ensemble import RandomForestRegressor
model_task3_gradient = RandomForestRegressor()
for label in labels3:
  scores = cross_val_score(model_task3_gradient,imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='r2')
  print(scores )

[0.42793697 0.42731861 0.40000982 0.42024863 0.41309528]
[0.60850106 0.62740096 0.62781769 0.5729509  0.60103628]
[0.34431017 0.35494033 0.28850139 0.37804934 0.33239153]
[0.64542098 0.64103194 0.61908753 0.62413833 0.650474  ]
Wall time: 12min 24s


In [48]:
for label in labels3:
  #model_task3 = Ridge(alpha=5e4, fit_intercept=False)
  # generated_models.append(deepcopy(model_task3.fit(imputed_data_slope_intersect, ground_truth[label])))
  generated_models.append(deepcopy(model_task3_gradient.fit(imputed_data_slope_intersect, ground_truth[label])))

In [55]:
print(test_features.shape)
num_test_features = test_features.shape[0]
num_test_prediction = int(num_test_features/num_hours)

imputed_test_features = imputer.transform(test_features)
imputed_test_features_slope_intersect = np.zeros((num_test_prediction, num_features*2+1))
test_features_id = np.zeros(num_test_prediction)

(151968, 37)


In [56]:
%%time
for i in range(num_test_prediction):
  current_hour = imputed_test_features[num_hours*i:num_hours*(i+1), 1]
  test_features_id[i] = imputed_test_features[num_hours*i, 0]
  for j in range(num_features):
    intersect, slope = Polynomial.fit(current_hour, imputed_test_features[num_hours*i:num_hours*(i+1), j+2], deg=1).coef
    imputed_test_features_slope_intersect[i, 0]       = current_hour[-1] # The total time of stay
    imputed_test_features_slope_intersect[i, 2*j+1]   = intersect
    imputed_test_features_slope_intersect[i, 2*(j+1)] = slope

Wall time: 50.8 s


In [57]:
len(generated_models)

15

In [58]:
prediction = [test_features_id]
for model in generated_models:
  prediction.append(model.predict(imputed_test_features_slope_intersect))

In [61]:
prediction = np.array(prediction)
prediction.shape

(16, 12664)

In [66]:
pd.DataFrame(prediction.T, columns=['pid'] + labels1 + labels2 + labels3).to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')

In [62]:
prediction.T[100]

array([1.02410000e+04, 7.20038188e-01, 1.05107540e-01, 2.06162531e-01,
       2.04650283e-01, 2.11885466e-01, 1.80464714e-01, 5.73128514e-01,
       2.31427837e-01, 2.16368329e-02, 6.94467991e-03, 1.58470000e+01,
       8.83870000e+01, 9.83400000e+01, 7.57460000e+01, 0.00000000e+00])

In [65]:
pd.DataFrame(prediction.T, columns=['pid'] + labels1 + labels2 + labels3).to_csv('prediction.csv', index=False, float_format='%.3f')